In [18]:
from backtesting import Backtest, Strategy
import yfinance as yf
import json
import pandas as pd
filemap = {
    "NYSE"  : "nyse_tickers.json",
    "NASDAQ": 'nasdaq_tickers.json',
    "SP500" : "S&P500_stocks.csv",
    "Yuanta": "Yuanta_us_stocks.csv"
}
def get_tickers(file="SP500"):
    if file in ["NYSE", "NASDAQ"]:
        f = open(filemap[file])
        dict = pd.Series(json.load(f))
    if file in ["SP500", "Yuanta"]:
        dict = pd.read_csv(filemap[file], header=None)
        dict = dict.iloc[:, 0]
    return dict


def WPR(data):
    period = 12
    max = data.Close.s.rolling(period).max()
    min = data.Close.s.rolling(period).min()
    wpr = (max - data.Close.s)/(max - min) * (-100) 
    return wpr

def moving_average_exp(data, period):
    return data.Close.s.ewm(period).mean()

def close_price(data):
    return data.Close.s
def open_price(data):
    return data.Open.s
# 如果下跌趨勢，當天開盤往上開=>做空
class WPR_2MA(Strategy):
    # parameters
    period_short_ma = 14
    period_long_ma = 28
    tp_factor = 1.1
    sl_factor = 0.95
    def init(self):
        period_long_ma = self.period_long_ma
        period_short_ma = self.period_short_ma
        self.short_ma = self.I(moving_average_exp, self.data, period=period_short_ma)
        self.long_ma = self.I(moving_average_exp, self.data, period=period_long_ma)
        self.close = self.I(close_price, self.data)
        self.open = self.I(open_price, self.data)
    def next(self):
        # paramaters
        tp_factor = self.tp_factor
        sl_factor = self.sl_factor
        # ma 
        long_ma0 = self.long_ma[-1]
        long_ma1 = self.long_ma[-2]
        # long_ma2 = self.long_ma[-3]
        short_ma0 = self.short_ma[-1]
        short_ma1 = self.short_ma[-2]
        # short_ma2 = self.short_ma[-3]
        bear_market = (long_ma0 > short_ma0) and (long_ma1 > short_ma1)
        bull_market = (long_ma0 < short_ma0) and (long_ma1 < short_ma1)

        close_price = self.close[-1]
        open = self.open[-1]
        previous_close = self.close[-2]

        sell_condition = (open - previous_close)/previous_close > 0.01
        buy_condition =  (previous_close - open)/previous_close > 0.01
        if self.trades:
            pass
        else:    
            if bull_market and buy_condition:
                self.buy(tp=close_price*tp_factor, sl=close_price*sl_factor)

            if bear_market and sell_condition:
                self.sell(tp=close_price*sl_factor, sl=close_price*tp_factor)


# =======================================
result = {}
backtest_list = get_tickers("SP500")[100:200]
backtest_list = ["OCGN"]
for ticker in backtest_list:
    df = yf.download(ticker.split(), start="2020-01-01", interval="1d", progress=False)
    try:
        bt = Backtest(df, WPR_2MA, commission=0.002, exclusive_orders=True)
        # stats, heatmap = bt.optimize(period_short_ma=[i for i in range(7, 22)],
        #         period_long_ma=[i for i in range(14, 29)], 
        #         maximize='Return [%]', max_tries=300,
        #         random_state=0,
        #         return_heatmap=True)
        # bt.plot()
        stats = bt.run()
        elements = [6, 7, 8, 10, 13, 17, 18, 23]
        result[ticker] = {stats.keys()[i] : stats.values[i] for i in elements}
    except ValueError:
        print(ticker + " doesn't exist, passed")
        pass
stats

Start                     2020-01-02 00:00:00
End                       2024-02-07 00:00:00
Duration                   1497 days 00:00:00
Exposure Time [%]                   52.034884
Equity Final [$]                  1229.586589
Equity Peak [$]                  11884.049211
Return [%]                         -87.704134
Buy & Hold Return [%]                3.718199
Return (Ann.) [%]                  -40.057911
Volatility (Ann.) [%]               33.912504
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -90.51623
Avg. Drawdown [%]                  -29.358321
Max. Drawdown Duration     1414 days 00:00:00
Avg. Drawdown Duration      365 days 00:00:00
# Trades                                  249
Win Rate [%]                        42.971888
Best Trade [%]                      14.670225
Worst Trade [%]                    -20.366523
Avg. Trade [%]                    

In [19]:
bt.plot()

c:\Users\p3749\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\p3749\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\p3749\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\Users\p3749\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p3238', ...)